## SNOTEL

In [21]:
import json
import requests
import pandas as pd
import geopandas as gpd
from sedona.spark import *

## Sedona Setup

In [23]:
config = SedonaContext.builder().\
      getOrCreate()

sedona = SedonaContext.create(config)

24/03/27 16:31:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/27 16:31:41 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
24/03/27 16:31:42 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to qjnq6fcbf1/spark-logs/spark-17b4e4ae18c64495b4f050527cf5aeac.inprogress. This is unsupported
24/03/27 16:32:06 WARN SedonaContext: Python files are not set. Sedona will not pre-load Python UDFs.


## SNOTEL Rest API

In [4]:
baseurl = "https://wcc.sc.egov.usda.gov/awdbRestApi/"

## Stations

parameters

In [5]:
payload = {
    'activeOnly': True,
    'returnStationElements': False
}

In [6]:
r = requests.get(baseurl + 'services/v1/stations', params=payload)

In [13]:
stations = r.json()

In [14]:
len(stations)

4363

In [16]:
df = pd.DataFrame(stations)

In [19]:
gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=gpd.points_from_xy(df.longitude, df.latitude))

In [20]:
gdf

,stationTriplet,stationId,stateCode,networkCode,name,dcoCode,countyName,huc,elevation,latitude,longitude,dataTimeZone,pedonCode,shefId,beginDate,endDate,geometry
0,2057:AL:SCAN,2057,AL,SCAN,AAMU-JTG,GC,Madison,060300020403,860.0,34.78333,-86.55000,-6.0,27979,AAMA1,2002-02-23 00:00:00.0,2100-01-01 00:00:00.0,POINT (-86.55000 34.78333)
1,ABY:CA:SNOW,ABY,CA,SNOW,Abbey,UN,Plumas,180201220103,5650.0,39.95500,-120.53800,NaN,NaN,NaN,1963-02-01 00:00:00.0,2100-01-01 00:00:00.0,POINT (-120.53800 39.95500)
2,0010:ID:COOP,0010,ID,COOP,Aberdeen Experimnt Stn,ID,Bingham,170402060704,4410.0,42.95000,-112.83333,NaN,NaN,ABDI1,1914-01-01 00:00:00.0,2100-01-01 00:00:00.0,POINT (-112.83333 42.95000)
3,1F01A:BC:SNOW,1F01A,BC,SNOW,Aberdeen Lake,OR,UNKNOWN,NaN,4298.0,50.14733,-119.05340,NaN,NaN,ABLQ2,1939-04-01 00:00:00.0,2100-01-01 00:00:00.0,POINT (-119.05340 50.14733)
4,0041:NM:COOP,0041,NM,COOP,Abiquiu Dam,UN,Rio Arriba,130201021005,6380.0,36.23333,-106.43333,NaN,NaN,ABIN5,1957-01-01 00:00:00.0,2100-01-01 00:00:00.0,POINT (-106.43333 36.23333)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4358,YBP:CA:SNOW,YBP,CA,SNOW,Yuba Pass,UN,Sierra,180201250202,6700.0,39.61700,-120.49200,NaN,NaN,NaN,1937-03-01 00:00:00.0,2100-01-01 00:00:00.0,POINT (-120.49200 39.61700)
4359,15356000:AK:USGS,15356000,AK,USGS,Yukon R at Eagle,AK,Southeast Fairbanks,190705051301,850.0,64.78917,-141.20009,NaN,NaN,NaN,1950-01-01 00:00:00.0,2100-01-01 00:00:00.0,POINT (-141.20009 64.78917)
4360,15453500:AK:USGS,15453500,AK,USGS,Yukon R nr Stevens Village,AK,Yukon-Koyukuk,190804040504,240.0,65.87510,-149.72035,NaN,NaN,NaN,1976-10-01 00:00:00.0,2100-01-01 00:00:00.0,POINT (-149.72035 65.87510)
4361,1033:CO:SNTL,1033,CO,SNTL,Zirkel,CO,Jackson,101800010202,9338.0,40.79492,-106.59544,-8.0,NaN,ZIRC2,2002-08-14 00:00:00.0,2100-01-01 00:00:00.0,POINT (-106.59544 40.79492)


In [31]:
station_df = sedona.createDataFrame(gdf)
station_df.createOrReplaceTempView('stations')

In [26]:
station_df.printSchema()

root
 |-- stationTriplet: string (nullable = true)
 |-- stationId: string (nullable = true)
 |-- stateCode: string (nullable = true)
 |-- networkCode: string (nullable = true)
 |-- name: string (nullable = true)
 |-- dcoCode: string (nullable = true)
 |-- countyName: string (nullable = true)
 |-- huc: string (nullable = true)
 |-- elevation: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- dataTimeZone: double (nullable = true)
 |-- pedonCode: string (nullable = true)
 |-- shefId: string (nullable = true)
 |-- beginDate: string (nullable = true)
 |-- endDate: string (nullable = true)
 |-- geometry: geometry (nullable = true)



In [25]:
station_df.show()

+---------------+---------+---------+-----------+--------------------+-------+------------+------------+---------+--------+----------+------------+---------+------+--------------------+--------------------+--------------------+
| stationTriplet|stationId|stateCode|networkCode|                name|dcoCode|  countyName|         huc|elevation|latitude| longitude|dataTimeZone|pedonCode|shefId|           beginDate|             endDate|            geometry|
+---------------+---------+---------+-----------+--------------------+-------+------------+------------+---------+--------+----------+------------+---------+------+--------------------+--------------------+--------------------+
|   2057:AL:SCAN|     2057|       AL|       SCAN|            AAMU-JTG|     GC|     Madison|060300020403|    860.0|34.78333|    -86.55|        -6.0|    27979| AAMA1|2002-02-23 00:00:...|2100-01-01 00:00:...|POINT (-86.55 34....|
|    ABY:CA:SNOW|      ABY|       CA|       SNOW|               Abbey|     UN|      Plum

In [32]:
sedona.sql("SELECT COUNT(*) FROM stations WHERE networkCode = 'SNTL'").show()

+--------+
|count(1)|
+--------+
|     899|
+--------+



## Data

parameters

* `beginDate` - yyyy-MM-dd HH:mm
* `centralTendencyType` - MEDIAN
* `duration` - DAILY
* `elements` - PREC, WTEQ, SMS:*, PREC::2
* `endDate` - yyyy-MM-dd HH:mm
* `stationTriplets` - *:*:*

In [58]:
payload = {
    'beginDate': '2024-03-26',
    'endDate': '2024-03-26',
    'centralTendencyType': 'MEDIAN',
    'duration': 'DAILY',
    'elements': 'WTEQ',
    'stationTriplets': '*:WA:SNTL'
}

In [59]:
r = requests.get(baseurl + 'services/v1/data', params=payload)

In [61]:
data = r.json()

In [62]:
df = pd.DataFrame(data)

In [63]:
df

,stationTriplet,data
0,908:WA:SNTL,"[{'stationElement': {'elementCode': 'WTEQ', 'o..."
1,990:WA:SNTL,"[{'stationElement': {'elementCode': 'WTEQ', 'o..."
2,352:WA:SNTL,"[{'stationElement': {'elementCode': 'WTEQ', 'o..."
3,1080:WA:SNTL,"[{'stationElement': {'elementCode': 'WTEQ', 'o..."
4,1107:WA:SNTL,"[{'stationElement': {'elementCode': 'WTEQ', 'o..."
...,...,...
70,832:WA:SNTL,"[{'stationElement': {'elementCode': 'WTEQ', 'o..."
71,841:WA:SNTL,"[{'stationElement': {'elementCode': 'WTEQ', 'o..."
72,974:WA:SNTL,"[{'stationElement': {'elementCode': 'WTEQ', 'o..."
73,909:WA:SNTL,"[{'stationElement': {'elementCode': 'WTEQ', 'o..."


In [73]:
data_list = [ (station['stationTriplet'], station['data'][0]['values'][0]['value'], station['data'][0]['values'][0].get('median'), station['data'][0]['values'][0]['date']) for station in data ]

In [76]:
df = pd.DataFrame(data_list, columns=['station', 'value', 'median', 'date'])

In [77]:
df

,station,value,median,date
0,908:WA:SNTL,34.1,49.2,2024-03-26
1,990:WA:SNTL,26.0,42.4,2024-03-26
2,352:WA:SNTL,8.1,13.0,2024-03-26
3,1080:WA:SNTL,33.2,62.8,2024-03-26
4,1107:WA:SNTL,32.2,56.4,2024-03-26
...,...,...,...,...
70,832:WA:SNTL,11.4,10.3,2024-03-26
71,841:WA:SNTL,10.1,11.2,2024-03-26
72,974:WA:SNTL,21.6,39.6,2024-03-26
73,909:WA:SNTL,21.8,33.6,2024-03-26


In [78]:
data_df = sedona.createDataFrame(df)

In [79]:
data_df.show()

+------------+-----+------+----------+
|     station|value|median|      date|
+------------+-----+------+----------+
| 908:WA:SNTL| 34.1|  49.2|2024-03-26|
| 990:WA:SNTL| 26.0|  42.4|2024-03-26|
| 352:WA:SNTL|  8.1|  13.0|2024-03-26|
|1080:WA:SNTL| 33.2|  62.8|2024-03-26|
|1107:WA:SNTL| 32.2|  56.4|2024-03-26|
| 375:WA:SNTL| 21.0|  26.0|2024-03-26|
| 376:WA:SNTL| 21.1|  26.4|2024-03-26|
| 942:WA:SNTL|  6.3|  17.6|2024-03-26|
|1109:WA:SNTL|  0.1|   1.2|2024-03-26|
|1085:WA:SNTL| 35.7|  59.6|2024-03-26|
| 418:WA:SNTL| 25.0|  34.1|2024-03-26|
| 420:WA:SNTL|  0.0|  13.0|2024-03-26|
| 943:WA:SNTL|  2.2|   8.6|2024-03-26|
| 998:WA:SNTL| 62.2|  91.9|2024-03-26|
| 910:WA:SNTL| 13.3|  37.7|2024-03-26|
| 478:WA:SNTL| 20.9|  28.6|2024-03-26|
|1159:WA:SNTL|  5.8|  10.1|2024-03-26|
|1256:WA:SNTL|  2.9|   NaN|2024-03-26|
| 502:WA:SNTL| 21.3|  23.2|2024-03-26|
| 507:WA:SNTL| 13.6|  18.8|2024-03-26|
+------------+-----+------+----------+
only showing top 20 rows



In [80]:
data_df.createOrReplaceTempView('snotel_data')

In [85]:
## Join Daily Observations 


snotel_data_df = sedona.sql("""
SELECT *, (value / median) AS pct_median
FROM stations
JOIN snotel_data 
WHERE snotel_data.station = stations.stationTriplet
""")

In [86]:
snotel_data_df.count()

75

In [89]:
snotel_data_df.show()

+--------------+---------+---------+-----------+-----------------+-------+----------+------------+---------+--------+----------+------------+---------+------+--------------------+--------------------+--------------------+------------+-----+------+----------+------------------+
|stationTriplet|stationId|stateCode|networkCode|             name|dcoCode|countyName|         huc|elevation|latitude| longitude|dataTimeZone|pedonCode|shefId|           beginDate|             endDate|            geometry|     station|value|median|      date|        pct_median|
+--------------+---------+---------+-----------+-----------------+-------+----------+------------+---------+--------+----------+------------+---------+------+--------------------+--------------------+--------------------+------------+-----+------+----------+------------------+
|   553:WA:SNTL|      553|       WA|       SNTL|        June Lake|     OR|  Skamania|170800020304|   3440.0|46.14778|-122.15413|        -8.0|      NaN| MRBW1|1980-10-

In [87]:
SedonaKepler.create_map(snotel_data_df)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/opt/conda/lib/python3.10/site-packages/jupyter_client/session.py:719: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(data={'unnamed': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,…

## Watershed - Spatial Join

In [ ]:
# TODO: load geojson, spatial join, calculate 